# Exploring and Cleaning the data

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('train_aug_in_kag_org.csv')

In [3]:
data.head(10)

,title,text,title_text,label
0,squirrel financial wellbeing,A Squirrel account makes it easy to stick to your budget and save. Start for free today!,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,senior insurance marketing,"Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.","senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,3i financial group inc.,"A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more","3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,brownstone insurance,"Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.","brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending,Learn more about Pacificwide Lending and our commercial & residential loan services in California.,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
5,city auto finance,"City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.","city auto finance. City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
6,"veterans financial, inc.","VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost about benefits","veterans financial, inc.. VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost 

In [4]:
data.shape

(34953, 4)

In [5]:
data.isnull().sum()

title         42
text           0
title_text     0
label          0
dtype: int64

In [6]:
data.dropna(inplace=True)
data.drop(columns=['text'], inplace=True)

In [7]:
data.head()

,title,title_text,label
0,squirrel financial wellbeing,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,senior insurance marketing,"senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,3i financial group inc.,"3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,brownstone insurance,"brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing


# Preparing our data for modeling 

In [8]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# title
title_corpus = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    title_corpus.append(row)

In [10]:
# text
text_corpus = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', data.iloc[i, 1]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus.append(row)

_Our Bag of Words_

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
title_cv = CountVectorizer(ngram_range=(1, 2), analyzer="word", max_features=200)   
X_title = title_cv.fit_transform(title_corpus).toarray()

In [13]:
X_title.shape

(34911, 200)

In [14]:
text_cv = CountVectorizer(ngram_range=(1, 2), analyzer="word", max_features=800)   
X_text = text_cv.fit_transform(text_corpus).toarray()

In [15]:
X_text.shape

(34911, 800)

In [16]:
X = np.concatenate((X_title, X_text), axis=1)
y = data.iloc[:, 2].values

In [17]:
X.shape, y.shape

((34911, 1000), (34911,))

In [18]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [19]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

# Splitting the data

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [21]:
X_train.shape, X_val.shape

((27928, 1000), (6983, 1000))

# Modeling

In [22]:
from time import time 

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier 

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

In [23]:
models = [
    ('GradientBoostingClassifier',GradientBoostingClassifier()),
    ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('LightGBM', LGBMClassifier())
]

results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 
                                  'Recall-MacroAVG', 'Recall-WeightedAVG', 'F1_score-MacroAVG','F1_score-WeightedAVG', 'Time_taken'])
for model_name, model in models:
    start = time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    end = time()
    
    record = {}
    record['Model'] = model_name
    record['Accuracy'] = accuracy_score(y_val, y_pred)*100
    record['Precision-MacroAVG'] = precision_score(y_val, y_pred, average='macro')*100
    record['Precision-WeightedAVG'] = precision_score(y_val, y_pred, average='weighted')*100
    record['Recall-MacroAVG'] = recall_score(y_val, y_pred, average='macro')*100
    record['Recall-WeightedAVG'] = recall_score(y_val, y_pred, average='weighted')*100
    record['F1_score-MacroAVG'] = f1_score(y_val, y_pred, average='macro')*100
    record['F1_score-WeightedAVG'] = f1_score(y_val, y_pred, average='weighted')*100
    record['Time_taken'] = end-start
    
    results = results.append(record, ignore_index=True)
    
results = results.sort_values(by='Accuracy', ascending=False)

In [24]:
results

,Model,Accuracy,Precision-MacroAVG,Precision-WeightedAVG,Recall-MacroAVG,Recall-WeightedAVG,F1_score-MacroAVG,F1_score-WeightedAVG,Time_taken
2,LightGBM,89.832450,86.282534,89.864140,81.892644,89.832450,83.701469,89.753656,55.910149
0,GradientBoostingClassifier,86.409852,80.408721,86.344015,75.531324,86.409852,77.236067,86.158179,1682.453250
1,XGBoost,84.734355,83.971844,84.727226,68.555169,84.734355,72.155929,84.115978,724.312357


In [25]:
# saving the models
import joblib

joblib.dump(models[0][1], 'GradientBoostingClassifier.sav')
joblib.dump(models[1][1], 'XGBoost.sav')
joblib.dump(models[2][1], 'LightGBM.sav')

['LightGBM.sav']

# Testing on test_data

In [26]:
test_data = pd.read_csv('test_in_kag_org.csv')

In [27]:
test_data.shape

(6501, 4)

In [28]:
test_data.isnull().sum()

title         0
text          0
title_text    0
label         0
dtype: int64

In [29]:
test_data['label'].nunique()

15

In [30]:
test_data.drop(columns=['text'], inplace=True)

# Preparing our data for modeling 

In [31]:
# title
title_corpus_test = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(test_data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', test_data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    title_corpus_test.append(row)

In [32]:
# text
text_corpus_test = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(test_data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', test_data.iloc[i, 1]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus_test.append(row)

In [33]:
len(text_corpus_test)

6501

In [34]:
text_corpus_test[0]

'centralized hub procore offer ability communicate team get full view project one place assign task stay connected google map google doc'

_Our Bag of Words_

In [35]:
X_title_test = title_cv.transform(title_corpus_test).toarray()

In [38]:
X_title_test.shape

(6501, 200)

In [36]:
X_text_test = text_cv.transform(text_corpus_test).toarray()

In [37]:
X_text_test.shape

(6501, 800)

In [39]:
X_test = np.concatenate((X_title_test, X_text_test), axis=1)
y_test = test_data.iloc[:, 2].values

In [40]:
X_test.shape, y_test.shape

((6501, 1000), (6501,))

In [41]:
y_test = le.transform(y_test)

In [42]:
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [43]:
import joblib

GB_model = joblib.load('GradientBoostingClassifier.sav')
XGB_model = joblib.load('XGBoost.sav') 
LGBM_model = joblib.load('LightGBM.sav')

In [44]:
models = [
    ('GradientBoostingClassifier',GB_model),
    ('XGBoost', XGB_model),
    ('LightGBM', LGBM_model)
]

results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 
                                  'Recall-MacroAVG', 'Recall-WeightedAVG', 'F1_score-MacroAVG','F1_score-WeightedAVG'])
for model_name, model in models:
    y_pred = model.predict(X_test)
    
    record = {}
    record['Model'] = model_name
    record['Accuracy'] = accuracy_score(y_test, y_pred)*100
    record['Precision-MacroAVG'] = precision_score(y_test, y_pred, average='macro')*100
    record['Precision-WeightedAVG'] = precision_score(y_test, y_pred, average='weighted')*100
    record['Recall-MacroAVG'] = recall_score(y_test, y_pred, average='macro')*100
    record['Recall-WeightedAVG'] = recall_score(y_test, y_pred, average='weighted')*100
    record['F1_score-MacroAVG'] = f1_score(y_test, y_pred, average='macro')*100
    record['F1_score-WeightedAVG'] = f1_score(y_test, y_pred, average='weighted')*100
    
    results = results.append(record, ignore_index=True)
    
results = results.sort_values(by='Accuracy', ascending=False)
results

,Model,Accuracy,Precision-MacroAVG,Precision-WeightedAVG,Recall-MacroAVG,Recall-WeightedAVG,F1_score-MacroAVG,F1_score-WeightedAVG
2,LightGBM,90.709122,72.445182,90.464356,67.668182,90.709122,68.408081,90.440937
1,XGBoost,89.001692,67.798057,88.868591,68.359958,89.001692,66.690976,88.710919
0,GradientBoostingClassifier,88.970928,67.247359,88.791859,67.993208,88.970928,66.788520,88.723922
